In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import KNNImputer
from sklearn import preprocessing, model_selection
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from itertools import combinations 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import os

2022-05-21 14:20:24.539027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-21 14:20:24.539101: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def greatest_class(prediction):
    max_item = max(prediction)
    return prediction.tolist().index(max_item)

def impute(x, y, imputer = None):
    if imputer:
        _x = pd.DataFrame(imputer.transform(x), columns = x.columns)
        return _x, y, imputer
    
    imputer = KNNImputer(n_neighbors=3)
    _x = pd.DataFrame(imputer.fit_transform(x), columns = x.columns)
    return _x, y, imputer

def oversample(x, y):
    x_smote, y_smote = SMOTETomek().fit_resample(x, y)
    return x_smote, y_smote

def select_rows(x, y, cols):
    df = pd.DataFrame(x, columns = cols)
    df = df[df.columns.drop(list(df.filter(regex='Positive discriminator|MTS|Hospitalisation')))]
    return df.to_numpy(), y, df.columns

def augment(x, y, augmenter = None):
    x = x.fillna(-1)
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('../data.csv')
dataset = df[(df['Respiratory rate'].notnull() | df['Heart rate'].notnull() | df['Temperature'].notnull() | df['Oxygen saturation'].notnull()) & df['Age'].notnull()]

In [6]:
data = dataset

In [7]:
fernando = data[data['Hospital '] == 'Fernando Fonseca Hospital']
juliana = data[data['Hospital '] == 'Juliana Childrens Hospital']
sophia = data[data['Hospital '] == 'Sophia Childrens Hospital']
mary = data[data['Hospital '] == 'St Mary']

In [8]:
def scale(x, y, scaler = None):
    if scaler:
        return pd.DataFrame(scaler.transform(x), columns = x.columns), y, scaler
    
    scaler = MinMaxScaler()
    norm_data = pd.DataFrame(scaler.fit_transform(x), columns = x.columns)

    return norm_data, y, scaler

In [9]:
def ohe(data):
    variables = data[['Presenting problem']]
    dummmies = pd.get_dummies(variables)
    df_ordinal_removed = data.drop(['Presenting problem'], axis=1)
    data = pd.concat([dummmies, df_ordinal_removed], axis=1)

    variables = data[['Positive discriminator']]
    dummmies = pd.get_dummies(variables)
    df_discriminator_removed = data.drop(['Positive discriminator'], axis=1)
    data = pd.concat([dummmies, df_discriminator_removed], axis=1)

    data['Hospitalisation'] = data['Hospitalisation'].replace({ 'no': 0, 'yes': 1})
    return data

In [10]:
def gen_folds(data, fold):
    num_folds = 10
    fold_no = 1
    kfold = model_selection.StratifiedKFold(n_splits=num_folds, shuffle=True)

    data = data.drop([
        'Patientnumber', 'Hospital ', 'Arrival date', 'MTS 1'
    ], axis = 1)
        
    X = data.drop(['Original MTS'], axis = 1)
    Y = data.filter(['Original MTS'])
    
    X = ohe(X)
    Y = Y['Original MTS'].replace({ 'Non urgent': 0, 'Standard': 1, 'Urgent': 2, 'Very urgent': 3, 'Emergent': 4})
       
    X, Y, _, columns = augment(X, Y)
    X[(X < 0) & (X != np.nan)] = np.nan
        
    for train, test in kfold.split(X, Y):
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')
        
        print('TRAIN: Normalizing data')
        x_train, y_train, escaler = scale(X.loc[train], Y.loc[train])
        print('TRAIN: Imputing')
        x_train, y_train, imputer = impute(x_train, y_train)
        print('TRAIN: Oversampling')
        x_train, y_train = oversample(x_train, y_train)
        print('TRAIN: Selecting rows')
        x_train, y_train, _ = select_rows(x_train, y_train, columns)
        
        print('TEST: Normalizing data')
        x_test, y_test, _ = scale(X.loc[test], Y.loc[test], escaler)
        print('TEST: Imputing')
        x_test, y_test, _ = impute(x_test, y_test, imputer)
        print('TEST: Selecting rows')
        x_test, y_test, filtered_columns = select_rows(x_test, y_test, columns)

        x_train_df = pd.DataFrame(x_train, columns = filtered_columns)
        y_train_df = pd.DataFrame(y_train.values, columns = ['MTS'])
        x_test_df  = pd.DataFrame(x_test, columns = filtered_columns)
        y_test_df  = pd.DataFrame(y_test.values, columns = ['MTS'])
        
        save_folds(x_train_df, y_train_df, x_test_df, y_test_df, fold, fold_no)
                
        fold_no = fold_no + 1

In [74]:
import random

In [11]:
def save_folds(x_train, y_train, x_test, y_test, fold, fold_no):
    path = 'folds/' + fold
    
    if not os.path.exists(path):
        os.makedirs(path)
        
    x_train.to_csv(path  + '/'  + str(fold_no) + '_x_train.csv', index=False)
    y_train.to_csv(path  + '/'  + str(fold_no) + '_y_train.csv', index=False)
    x_test.to_csv(path  + '/'  + str(fold_no) + '_x_test.csv', index=False)
    y_test.to_csv(path  + '/'  + str(fold_no) + '_y_test.csv', index=False)

In [84]:
def run_model(label, fold, model):
    num_folds = 10
    ids = list(range(1, num_folds + 1))
#    random.shuffle(ids)
    
    for i in ids:
        x_train = pd.read_csv('folds/' + fold + '/' + str(i) + '_x_train.csv', index_col=False).to_numpy()
        y_train = pd.read_csv('folds/' + fold + '/' + str(i) + '_y_train.csv', index_col=False).to_numpy()
        x_test  = pd.read_csv('folds/' + fold + '/' + str(i) + '_x_test.csv', index_col=False).to_numpy()
        y_test  = pd.read_csv('folds/' + fold + '/' + str(i) + '_y_test.csv', index_col=False).to_numpy()
            
        y_train = np.ravel(y_train, order = 'C')
        y_test = np.ravel(y_test, order = 'C')
         
        print('------------------------------------------------------------------------')
        print(f'Fold {i} ...')
        model(x_train, y_train, x_test, y_test, label, i)

In [88]:
def svm(x_train, y_train, x_test, y_test, label, fold_no):
    print('Fitting model')
    clf = SVC(decision_function_shape='ovr', kernel='rbf', C=100, gamma=100)
    clf.fit(x_train, y_train)

    print('Predicting')
    predicted = clf.decision_function(x_test)

    class_per_prediction = list(map(greatest_class, predicted))
    save_to_file([class_per_prediction, y_test], label, fold_no)

In [89]:
def mlp(x_train, y_train, x_test, y_test, label, fold_no):
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1],)))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(5, activation = 'softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

    print('Fitting model')
    history = model.fit(x_train, y_train, batch_size=64, epochs=50)
    scores = model.evaluate(x_test, y_test, verbose=0)

    print('Predicting')
    predicted = model.predict(x_test)

    class_per_prediction = list(map(greatest_class, predicted))
    save_to_file([class_per_prediction, y_test], label, fold_no)

In [90]:
def gb(x_train, y_train, x_test, y_test, label, fold_no):
    print('Fitting model')
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=5)
    clf.fit(x_train, y_train)

    print('Predicting')
    score = clf.score(x_test, y_test)
    print(f'Score for fold {fold_no}: {score}')

    predicted = clf.predict(x_test)
    save_to_file([predicted, y_test], label, fold_no)

In [91]:
def ab(x_train, y_train, x_test, y_test, label, fold_no):
    print('Fitting model')
    clf = AdaBoostClassifier(n_estimators=100)
    clf.fit(x_train, y_train)

    print('Predicting')
    score = clf.score(x_test, y_test)
    print(f'Score for fold {fold_no}: {score}')

    predicted = clf.predict(x_test)
    save_to_file([predicted, y_test], label, fold_no)

In [92]:
def rf(x_train, y_train, x_test, y_test, label, fold_no):
    print('Fitting model')
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(x_train, y_train)

    print('Predicting')
    score = clf.score(x_test, y_test)
    print(f'Score for fold {fold_no}: {score}')

    predicted = clf.predict(x_test)
    save_to_file([predicted, y_test], label, fold_no)

In [93]:
def knn(x_train, y_train, x_test, y_test, label, fold_no):
    print('Fitting model')
    clf = KNeighborsClassifier(n_neighbors=10)
    clf.fit(x_train, y_train)

    print('Predicting')
    score = clf.score(x_test, y_test)
    print(f'Score for fold {fold_no}: {score}')

    predicted = clf.predict(x_test)
    save_to_file([predicted, y_test], label, fold_no)

In [94]:
def dt(x_train, y_train, x_test, y_test, label, fold_no):
    print('Fitting model')
    clf = DecisionTreeClassifier(max_depth = 10, min_samples_split = 5, min_samples_leaf = 5)
    clf.fit(x_train, y_train)

    print('Predicting')
    score = clf.score(x_test, y_test)
    print(f'Score for fold {fold_no}: {score}')

    predicted = clf.predict(x_test)
    save_to_file([predicted, y_test], label, fold_no)

In [95]:
def save_to_file(predictions, tag, fold_no):
    path = 'results/' + tag + '/'
    
    if not os.path.exists(path):
        os.makedirs(path)    

    with open(path + str(fold_no) + '-predicted.txt', 'w+') as writer:
        writer.writelines(list(map(str, predictions[0])))
        writer.write("\n")
            
    with open(path + str(fold_no) + '-label.txt', 'w+') as writer:
        writer.writelines(list(map(str, predictions[1])))
        writer.write("\n")

# Run tests

In [97]:
for i in range(30):
    gen_folds(pd.concat([juliana]).reset_index(drop=True), 'juliana/' + str(i))
    run_model('juliana-svm/' + str(i), 'juliana/' + str(i), svm)
    run_model('juliana-rf/' + str(i), 'juliana/' + str(i), rf)
    run_model('juliana-mlp/' + str(i), 'juliana/' + str(i), mlp)
    run_model('juliana-ab/' + str(i), 'juliana/' + str(i), ab)
    run_model('juliana-knn/' + str(i), 'juliana/' + str(i), knn)
    run_model('juliana-gb/' + str(i), 'juliana/' + str(i), gb)
    run_model('juliana-dt/' + str(i), 'juliana/' + str(i), dt)
    print("============================ EXPERIMENT " + str(i) + " =========================\n")

------------------------------------------------------------------------
Training for fold 1 ...
TRAIN: Normalizing data
TRAIN: Imputing
TRAIN: Oversampling
TRAIN: Selecting rows
TEST: Normalizing data
TEST: Imputing
TEST: Selecting rows
------------------------------------------------------------------------
Training for fold 2 ...
TRAIN: Normalizing data
TRAIN: Imputing
TRAIN: Oversampling
TRAIN: Selecting rows
TEST: Normalizing data
TEST: Imputing
TEST: Selecting rows
------------------------------------------------------------------------
Training for fold 3 ...
TRAIN: Normalizing data
TRAIN: Imputing
TRAIN: Oversampling
TRAIN: Selecting rows
TEST: Normalizing data
TEST: Imputing
TEST: Selecting rows
------------------------------------------------------------------------
Training for fold 4 ...
TRAIN: Normalizing data
TRAIN: Imputing
TRAIN: Oversampling
TRAIN: Selecting rows
TEST: Normalizing data
TEST: Imputing
TEST: Selecting rows
------------------------------------------------

108/108 [==============================] - 0s 1ms/step - loss: 0.3180 - accuracy: 0.8785
Epoch 33/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3164 - accuracy: 0.8817
Epoch 34/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3116 - accuracy: 0.8797
Epoch 35/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3066 - accuracy: 0.8839
Epoch 36/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3006 - accuracy: 0.8865
Epoch 37/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3014 - accuracy: 0.8865
Epoch 38/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3012 - accuracy: 0.8817
Epoch 39/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2876 - accuracy: 0.8910
Epoch 40/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2870 - accuracy: 0.8897
Epoch 41/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2851 - accuracy: 0.8956
Ep

109/109 [==============================] - 0s 1ms/step - loss: 0.5329 - accuracy: 0.7922
Epoch 12/50
109/109 [==============================] - 0s 1ms/step - loss: 0.5480 - accuracy: 0.7849
Epoch 13/50
109/109 [==============================] - 0s 1ms/step - loss: 0.5074 - accuracy: 0.8043
Epoch 14/50
109/109 [==============================] - 0s 2ms/step - loss: 0.4965 - accuracy: 0.8066
Epoch 15/50
109/109 [==============================] - 0s 2ms/step - loss: 0.4802 - accuracy: 0.8110
Epoch 16/50
109/109 [==============================] - 0s 2ms/step - loss: 0.4708 - accuracy: 0.8182
Epoch 17/50
109/109 [==============================] - 0s 2ms/step - loss: 0.4632 - accuracy: 0.8208
Epoch 18/50
109/109 [==============================] - 0s 2ms/step - loss: 0.4509 - accuracy: 0.8248
Epoch 19/50
109/109 [==============================] - 0s 2ms/step - loss: 0.4465 - accuracy: 0.8244
Epoch 20/50
109/109 [==============================] - 0s 2ms/step - loss: 0.4323 - accuracy: 0.8315
Ep

108/108 [==============================] - 0s 2ms/step - loss: 0.2973 - accuracy: 0.8884
Epoch 42/50
108/108 [==============================] - 0s 3ms/step - loss: 0.2951 - accuracy: 0.8902
Epoch 43/50
108/108 [==============================] - 0s 2ms/step - loss: 0.2943 - accuracy: 0.8867
Epoch 44/50
108/108 [==============================] - 0s 2ms/step - loss: 0.2902 - accuracy: 0.8925
Epoch 45/50
108/108 [==============================] - 0s 3ms/step - loss: 0.2844 - accuracy: 0.8915
Epoch 46/50
108/108 [==============================] - 0s 2ms/step - loss: 0.2841 - accuracy: 0.8944
Epoch 47/50
108/108 [==============================] - 0s 2ms/step - loss: 0.2749 - accuracy: 0.8964
Epoch 48/50
108/108 [==============================] - 0s 2ms/step - loss: 0.2765 - accuracy: 0.8981
Epoch 49/50
108/108 [==============================] - 0s 2ms/step - loss: 0.2747 - accuracy: 0.8965
Epoch 50/50
108/108 [==============================] - 0s 2ms/step - loss: 0.2747 - accuracy: 0.8970
Pr

108/108 [==============================] - 0s 1ms/step - loss: 0.4072 - accuracy: 0.8515
Epoch 21/50
108/108 [==============================] - 0s 1ms/step - loss: 0.4007 - accuracy: 0.8462
Epoch 22/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3941 - accuracy: 0.8535
Epoch 23/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3865 - accuracy: 0.8567
Epoch 24/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3849 - accuracy: 0.8565
Epoch 25/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3750 - accuracy: 0.8606
Epoch 26/50
108/108 [==============================] - 0s 3ms/step - loss: 0.3663 - accuracy: 0.8659
Epoch 27/50
108/108 [==============================] - 0s 2ms/step - loss: 0.3604 - accuracy: 0.8675
Epoch 28/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3602 - accuracy: 0.8677
Epoch 29/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3459 - accuracy: 0.8768
Ep

...

108/108 [==============================] - 0s 1ms/step - loss: 0.3078 - accuracy: 0.8840
Epoch 38/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3026 - accuracy: 0.8884
Epoch 39/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3024 - accuracy: 0.8887
Epoch 40/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2968 - accuracy: 0.8876
Epoch 41/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2919 - accuracy: 0.8903
Epoch 42/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2930 - accuracy: 0.8892
Epoch 43/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2831 - accuracy: 0.8921
Epoch 44/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2803 - accuracy: 0.8954
Epoch 45/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2772 - accuracy: 0.8935
Epoch 46/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2819 - accuracy: 0.8955
Ep

108/108 [==============================] - 0s 1ms/step - loss: 0.4472 - accuracy: 0.8269
Epoch 17/50
108/108 [==============================] - 0s 1ms/step - loss: 0.4270 - accuracy: 0.8354
Epoch 18/50
108/108 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.8363
Epoch 19/50
108/108 [==============================] - 0s 1ms/step - loss: 0.4093 - accuracy: 0.8446
Epoch 20/50
108/108 [==============================] - 0s 1ms/step - loss: 0.4018 - accuracy: 0.8466
Epoch 21/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3955 - accuracy: 0.8480
Epoch 22/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3874 - accuracy: 0.8483
Epoch 23/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3821 - accuracy: 0.8548
Epoch 24/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3753 - accuracy: 0.8625
Epoch 25/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3710 - accuracy: 0.8577
Ep

108/108 [==============================] - 0s 1ms/step - loss: 0.2835 - accuracy: 0.8938
Epoch 47/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2803 - accuracy: 0.8947
Epoch 48/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2802 - accuracy: 0.8911
Epoch 49/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2753 - accuracy: 0.8953
Epoch 50/50
108/108 [==============================] - 0s 1ms/step - loss: 0.2632 - accuracy: 0.8988
Predicting
------------------------------------------------------------------------
Fold 9 ...
Fitting model
Epoch 1/50
108/108 [==============================] - 0s 1ms/step - loss: 1.1237 - accuracy: 0.5011
Epoch 2/50
108/108 [==============================] - 0s 1ms/step - loss: 0.8661 - accuracy: 0.6297
Epoch 3/50
108/108 [==============================] - 0s 1ms/step - loss: 0.7543 - accuracy: 0.6887
Epoch 4/50
108/108 [==============================] - 0s 1ms/step - loss: 0.6963 - accuracy: 0.722

108/108 [==============================] - 0s 1ms/step - loss: 0.3668 - accuracy: 0.8614
Epoch 26/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3544 - accuracy: 0.8685
Epoch 27/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3575 - accuracy: 0.8657
Epoch 28/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3478 - accuracy: 0.8681
Epoch 29/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3375 - accuracy: 0.8743
Epoch 30/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3288 - accuracy: 0.8814
Epoch 31/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3292 - accuracy: 0.8781
Epoch 32/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3268 - accuracy: 0.8773
Epoch 33/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3217 - accuracy: 0.8795
Epoch 34/50
108/108 [==============================] - 0s 1ms/step - loss: 0.3139 - accuracy: 0.8830
Ep

Predicting
Score for fold 10: 0.6666666666666666
============================ EXPERIMENT 29 =========================

